In [531]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests


import folium
from folium.plugins import HeatMap
import geopandas as gpd
import contextily as ctx

from dotenv import load_dotenv
import os
import math
from haversine import haversine
from sklearn.neighbors import BallTree


In [ ]:
df_starbucks = pd.read_csv('./data/df_starbucks.csv', encoding='utf-8-sig')
df_seoul = pd.read_csv('./data/df_seoul.csv', encoding='utf-8-sig')
df_population = pd.read_csv('./data/df_population.csv', encoding='utf-8-sig')
df_building_price = pd.read_csv('./data/df_building.csv', encoding='utf-8-sig')

df_subway = pd.read_csv('./data/df_subway.csv', encoding='utf-8-sig')
df_subway_on = pd.read_csv('./data/df_subway_on.csv', encoding='utf-8-sig')
df_subway_off = pd.read_csv('./data/df_subway_off.csv', encoding='utf-8-sig')



In [479]:
df_seoul[df_seoul['상권업종소분류명'] == '카페'].head(1).T

,0
상가업소번호,MA010120220800000033
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
시도명,서울특별시
시군구명,종로구
행정동코드,11110540
행정동명,삼청동
법정동명,삼청동


In [508]:
df_starbucks.head(1).T

,0
상호명,아크로힐스논현
위도,37.509013
경도,127.040352
행정동명,역삼동
시군구명,강남구
도로명주소,서울특별시 강남구 봉은사로 304
지번주소,서울특별시 강남구 역삼동 681
스타벅스,1
상권업종대분류명,음식
상권업종중분류명,비알코올


In [ ]:
# 거리 구하는 방법1: haversine 
haversine((37.517473,127.0411685), (37.5102747, 127.0438167), unit = 'km')

# 거리 구하는 방법2: euclidean
def euclidean_distance_from_latlon(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (미터)

    # 위경도를 라디안으로 변환
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # 직교 좌표로 변환
    x1 = R * math.cos(lat1_rad) * math.cos(lon1_rad)
    y1 = R * math.cos(lat1_rad) * math.sin(lon1_rad)
    z1 = R * math.sin(lat1_rad)
    x2 = R * math.cos(lat2_rad) * math.cos(lon2_rad)
    y2 = R * math.cos(lat2_rad) * math.sin(lon2_rad)
    z2 = R * math.sin(lat2_rad)

    # 거리 계산
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return distance

# 거리 구하는 방법3: BallTree
def cnt_stores(df, df_store, distance=0.7):
    # 1. df_store (음식점 등)을 기준으로 BallTree 구성
    df_store_coords = np.radians(df_store[['위도', '경도']])
    tree = BallTree(df_store_coords, metric='haversine')

    # 2. 위경도
    df_coords = np.radians(df[['위도', '경도']])
    radius = distance / 6371  # 거리 km → 라디안

    # 3. 각 기준점의 반경 내 음식점 개수
    counts = tree.query_radius(df_coords, r=radius, count_only=True)

    return counts  # 배열 반환: df의 각 로우마다 음식점 수

# 거리 구하는 방법4: balltree + 라디안

def closest_radian(df1, df2, df2_col, lat_col='위도', lon_col='경도'):
    # 좌표를 라디안으로 변환
    df1_coords = np.radians(df1[[lat_col, lon_col]].to_numpy())
    df2_coords = np.radians(df2[[lat_col, lon_col]].to_numpy())

    tree = BallTree(df2_coords, metric='haversine')

    # 각 df1 좌표에 대해 가장 가까운 df2 좌표 인덱스 및 거리 추출
    distances, indices = tree.query(df1_coords, k=1)

    # 거리: 라디안 → km (지구 반지름 6371km 사용)
    distances_km = distances.flatten() * 6371

    # 가장 가까운 df2의 특정 컬럼 값 추출
    closest_values = df2.iloc[indices.flatten()][df2_col].reset_index(drop=True)

    # 반환: 거리(km), df2_col
    return distances_km, closest_values

In [7]:
df_seoul.head(1).T

,0
상가업소번호,MA010120220800000033
상호명,부동산임대김은숙
지점명,NaN
상권업종대분류코드,I2
상권업종대분류명,음식
상권업종중분류코드,I212
상권업종중분류명,비알코올
상권업종소분류코드,I21201
상권업종소분류명,카페
표준산업분류코드,I56229


In [8]:
df_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540517 entries, 0 to 540516
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     540517 non-null  object 
 1   상호명        540517 non-null  object 
 2   지점명        48544 non-null   object 
 3   상권업종대분류코드  540517 non-null  object 
 4   상권업종대분류명   540517 non-null  object 
 5   상권업종중분류코드  540517 non-null  object 
 6   상권업종중분류명   540517 non-null  object 
 7   상권업종소분류코드  540517 non-null  object 
 8   상권업종소분류명   540517 non-null  object 
 9   표준산업분류코드   540317 non-null  object 
 10  표준산업분류명    540317 non-null  object 
 11  시도코드       540517 non-null  int64  
 12  시도명        540517 non-null  object 
 13  시군구코드      540517 non-null  int64  
 14  시군구명       540517 non-null  object 
 15  행정동코드      540517 non-null  int64  
 16  행정동명       540517 non-null  object 
 17  법정동코드      540517 non-null  int64  
 18  법정동명       540517 non-null  object 
 19  지번코드       540517 non-n

In [12]:
df_population.head(1)

,행정기관코드,기준연월,시도명,시군구명,읍면동명,계,0세_6세,7세_19세,20세_25세,26세_30세,31세_40세,41세_50세,51세_60세,61세_109세,읍면동/구,위도,경도
0,1111051500,2025-05-31,서울특별시,종로구,청운효자동,10978,309,1239,680,797,1486,1775,1849,2843,청운효자동,37.584137,126.970652


In [14]:
df = df_seoul.copy()
df = df[['상가업소번호', '상호명', '상권업종대분류명', '상권업종중분류명','상권업종소분류명','표준산업분류명','행정동코드','시도코드','법정동코드','시도명','행정동명','법정동명','건물명','경도','위도']]
df = df.set_index('상가업소번호')

df.head(1)

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,행정동코드,시도코드,법정동코드,시도명,행정동명,법정동명,건물명,경도,위도
상가업소번호,,,,,,,,,,,,,,
MA010120220800000033,부동산임대김은숙,음식,비알코올,카페,기타 비알코올 음료점업,11110540,11,1111014000,서울특별시,삼청동,삼청동,NaN,126.98184,37.58625


In [18]:
# 1. 700m 내, 음식점, 카페 수
# 음식, 카페 df 분리
df_restaurant = df[df['상권업종대분류명']=='음식']
df_cafe = df[df['상권업종소분류명']=='카페']
print(df.shape)
print(df_restaurant.shape)
print(df_cafe.shape)

# df 로우별로 근방 700내 가게 수
cnt_store_nearby = cnt_stores(df, df, distance=0.7)
print(f"cnt_store_nearby:{cnt_store_nearby}")

# df 로우별로 근방 700내 레스토랑 수
cnt_restaurant_nearby = cnt_stores(df, df_restaurant, distance=0.7)
print(f"cnt_restaurant_nearby:{cnt_restaurant_nearby}")

# df 로우별로 근방 700내 카페 수
cnt_cafe_nearby = cnt_stores(df, df_cafe, distance=0.7)
print(f"cnt_cafe_nearby:{cnt_cafe_nearby}")



(540517, 14)
(139929, 14)
(22945, 14)
cnt_store_nearby:[ 638 7811 1681 ... 5076 2542 1668]
cnt_restaurant_nearby:[ 153 1477  408 ...  927  462  629]
cnt_cafe_nearby:[ 59 261  51 ... 153  72 116]


In [21]:
df['근방가게수'] = cnt_store_nearby
df['근방음식점수'] = cnt_restaurant_nearby
df['근방카페수'] = cnt_cafe_nearby

In [ ]:
# 2. 가장 가까운 지하철 역, 거리, 지하철 승하차 수 구하기

closest_station = closest_radian(df1=df, df2=df_subway, lat_col='위도', lon_col='경도', df2_col='역명')
station_distance, station_name = closest_station

In [37]:
df['근접역'] = station_name.values
df['근접역거리'] = station_distance
df.head(1).T

상가업소번호,MA010120220800000033
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
표준산업분류명,기타 비알코올 음료점업
행정동코드,11110540
시도코드,11
법정동코드,1111014000
시도명,서울특별시
행정동명,삼청동


In [38]:
df.isna().sum()

상호명              0
상권업종대분류명         0
상권업종중분류명         0
상권업종소분류명         0
표준산업분류명        200
행정동코드            0
시도코드             0
법정동코드            0
시도명              0
행정동명             0
법정동명             0
건물명         275503
경도               0
위도               0
근방가게수            0
근방음식점수           0
근방카페수            0
근접역              0
근접역거리            0
dtype: int64

In [56]:
df_subway_ride = pd.merge(left=df_subway_on,
                          right=df_subway_off,
                          how='inner',
                          on=['역명','호선명'])
print(df_subway_on.shape)
print(df_subway_off.shape)
print(df_subway_ride.shape)

(286, 5)
(286, 5)
(286, 8)


In [60]:
# 호선이 여러개인 경우 역명 중복 출력 --> sum
df_subway_ride = df_subway_ride.groupby('역명')[['승차_이용객수','승차_출근시간대','승차_퇴근시간대', '하차_이용객수', '하차_출근시간대','하차_퇴근시간대']].agg('sum').reset_index()
df_subway_ride.shape

(251, 7)

In [69]:
merged_df = pd.merge(left=df,
                        right=df_subway_ride,
                        how='inner',
                        left_on='근접역',
                        right_on='역명')

In [70]:
merged_df.head(1).T

,0
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
표준산업분류명,기타 비알코올 음료점업
행정동코드,11110540
시도코드,11
법정동코드,1111014000
시도명,서울특별시
행정동명,삼청동


In [97]:
df = merged_df.copy()

In [72]:
# 행정동 인구 merge: key = 행정동코드
df_population.head(1)

,행정기관코드,기준연월,시도명,시군구명,읍면동명,계,0세_6세,7세_19세,20세_25세,26세_30세,31세_40세,41세_50세,51세_60세,61세_109세,읍면동/구,위도,경도
0,1111051500,2025-05-31,서울특별시,종로구,청운효자동,10978,309,1239,680,797,1486,1775,1849,2843,청운효자동,37.584137,126.970652


In [92]:
print(len(df['행정동코드'][0].astype('str')))
print(df['행정동코드'].dtype)
print(len(df_population['행정기관코드'][0].astype('str')))
print(df_population['행정기관코드'].dtype)

8
int64
10
int64


/var/folders/q2/0w8s3xgj6szdvf0r9_0lyqfr0000gn/T/ipykernel_50336/1894338032.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(len(df['행정동코드'][0].astype('str')))


In [104]:
df_population = pd.read_csv('df_population.csv', encoding='utf-8-sig')

df_population['행정기관코드'] = df_population['행정기관코드'].astype('str')
df_population['행정기관코드'] = df_population['행정기관코드'].str[:8]

df['행정동코드'] = df['행정동코드'].astype('str')

print(f"df_population:{df_population.shape}")
print(f"df:{df.shape}")


df_merged = pd.merge(left=df,
                     right=df_population.drop(columns=['시도명', '시군구명','읍면동명','읍면동/구','위도','경도'], axis=1),
                     how='inner',
                     left_on='행정동코드',
                     right_on='행정기관코드')

print(f"df_merged:{df_merged.shape}")

df_population:(426, 17)
df:(400453, 26)
df_merged:(400453, 37)


In [111]:
df = df_merged.copy()

In [113]:
df_building_price.head(1)

,시군구,지번,도로명,용도지역,건축물주용도,전용/연면적(㎡),거래금액(만원),주소,위도,경도
0,서울특별시 동작구 노량진동,283-3,노량진로6길,제3종일반주거,제2종근린생활,9.88,"7,300",서울특별시 동작구 노량진동 283-3,37.510849,126.936012


In [ ]:
df_building_price['거래금액(만원)'] = df_building_price['거래금액(만원)'].str.replace(',','')
df_building_price['거래금액(만원)'] = df_building_price['거래금액(만원)'].astype('int')
df_building_price['평당거래금액(만원)'] = df_building_price['거래금액(만원)'] / df_building_price['전용/연면적(㎡)'] / 3.3058
df_building_price['평당거래금액(만원)'] = df_building_price['평당거래금액(만원)'].round(1)
df_building_price.head(1)

,시군구,지번,도로명,용도지역,건축물주용도,전용/연면적(㎡),거래금액(만원),주소,위도,경도,평당거래금액(만원)
0,서울특별시 동작구 노량진동,283-3,노량진로6길,제3종일반주거,제2종근린생활,9.88,7300,서울특별시 동작구 노량진동 283-3,37.510849,126.936012,223.5


In [119]:
distance, price = closest_radian(df1=df, df2=df_building_price, lat_col='위도', lon_col='경도', df2_col='평당거래금액(만원)')

In [121]:
distance

array([0.20117832, 0.00203155, 0.02360837, ..., 0.190689  , 0.09640539,
       0.20447678], shape=(400453,))

In [124]:
df['평당거래금액(만원)'] = price.values
df.head(1).T

,0
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
표준산업분류명,기타 비알코올 음료점업
행정동코드,11110540
시도코드,11
법정동코드,1111014000
시도명,서울특별시
행정동명,삼청동


In [125]:
df.isna().sum()

상호명                0
상권업종대분류명           0
상권업종중분류명           0
상권업종소분류명           0
표준산업분류명          143
행정동코드              0
시도코드               0
법정동코드              0
시도명                0
행정동명               0
법정동명               0
건물명           200942
경도                 0
위도                 0
근방가게수              0
근방음식점수             0
근방카페수              0
근접역                0
근접역거리              0
역명                 0
승차_이용객수            0
승차_출근시간대           0
승차_퇴근시간대           0
하차_이용객수            0
하차_출근시간대           0
하차_퇴근시간대           0
행정기관코드             0
기준연월               0
계                  0
0세_6세              0
7세_19세             0
20세_25세            0
26세_30세            0
31세_40세            0
41세_50세            0
51세_60세            0
61세_109세           0
평당거래금액(만원)         0
dtype: int64

In [127]:
df = df.dropna(axis=1)
df.to_csv('store_preprocessed.csv', encoding='utf-8-sig', index=False)